In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime

pd.set_option("display.max_columns", 100)
start_time = datetime.datetime.now()

c:\Users\rsirupa\.conda\envs\asim_baydag\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
### Run Time Function
def print_runtime(t1, t2):
    tot_sec = t2.timestamp()-t1.timestamp()
    hours = tot_sec//3600
    minutes = (tot_sec-hours*3600)//60
    seconds = tot_sec-hours*3600-minutes*60

    print("Run Time:", hours, 'hrs', minutes, 'mins', round(seconds), "sec")
    
    return

In [3]:
# Define Data Paths
### Households file
hh_loc = "C:/abm_runs/rohans/output/final_households.csv"

### Census data 
acs_loc = "C:/abm_runs/rohans/calibration/auto_ownership/targets/sandag_ao_data_tract_2021_5yr.csv"
base_target_loc = "C:/abm_runs/rohans/calibration/auto_ownership/targets/ao_targets_2021.csv"

### Shape files
maz_loc = "C:/abm_runs/rohans/calibration/shp/mgra15/mgra15.shp"
tract_loc = "C:/abm_runs/rohans/calibration/shp/ca_2022_tracts/tl_2022_06_tract.shp"

### Location to save summaries
output_loc = "C:/abm_runs/rohans/calibration/auto_ownership/tract_level_summary"

### Crosswalk location
xwalk_loc = "C:/abm_runs/rohans/calibration/auto_ownership/tract_level_summary/maz_tract_xwalk.csv"

# Parameters
### if create_xwalk is '1' a new crosswalk will be created
### else the script will read xwalk file defined above as 'xwalk_loc'
create_xwalk = 0
xwalk_file_name = 'maz_tract_xwalk.csv'

In [4]:
### Read files
acs_df = pd.read_csv(acs_loc)
hh_orig_df = pd.read_csv(hh_loc)
maz_gdf = gpd.read_file(maz_loc)
tract_gdf = gpd.read_file(tract_loc)
base_share = pd.read_csv(base_target_loc, index_col=0)

### Calcuate weight of each household
hh_orig_df['weight'] = 1/hh_orig_df['sample_rate']

### Remove group quarter (gq) households
hh_df = hh_orig_df[hh_orig_df['HHT'].isin([1, 2, 3, 4, 5, 6, 7])]
hh_gq_df = hh_orig_df[hh_orig_df['HHT']==0]
# hh_df = hh_orig_df.copy()

### Change shape files CRS to 4326 (Latitude and Longitude System)
maz_gdf = maz_gdf.to_crs(4326)
tract_gdf = tract_gdf.to_crs(4326)

In [5]:
### Calculate asim shares
asim_freq = pd.pivot_table(hh_orig_df, index='auto_ownership', values='weight', aggfunc=np.sum)
asim_share = asim_freq/asim_freq.sum()

### Calculate calibration constants
ao_summary_no_gq = base_share.rename({'share': 'census'}, axis=1)
ao_summary_no_gq['model'] = asim_share['weight']
ao_summary_no_gq['ratio'] = ao_summary_no_gq['model']/ao_summary_no_gq['census']

ao_summary_no_gq.to_csv('C:/abm_runs/rohans/calibration/auto_ownership/ao_summary_with_gq.csv')
ao_summary_no_gq

,census,model,ratio
auto_ownership,,,
0,0.054026,0.122970,2.276126
1,0.295868,0.295393,0.998396
2,0.398939,0.357247,0.895492
3,0.160256,0.143520,0.895567
4,0.090911,0.080870,0.889551


In [6]:
### Calculate asim shares
asim_freq = pd.pivot_table(hh_df, index='auto_ownership', values='weight', aggfunc=np.sum)
asim_share = asim_freq/asim_freq.sum()

### Calculate calibration constants
ao_summary = base_share.rename({'share': 'census'}, axis=1)
ao_summary['model'] = asim_share['weight']
ao_summary['ratio'] = ao_summary['model']/ao_summary['census']

ao_summary.to_csv('C:/abm_runs/rohans/calibration/auto_ownership/ao_summary_without_gq.csv')
ao_summary

,census,model,ratio
auto_ownership,,,
0,0.054026,0.053712,0.994189
1,0.295868,0.306149,1.034748
2,0.398939,0.393179,0.985563
3,0.160256,0.157956,0.985645
4,0.090911,0.089004,0.979024


In [7]:
### Calculate asim shares
asim_freq = pd.pivot_table(hh_gq_df, index='auto_ownership', values='weight', aggfunc=np.sum)
asim_share = asim_freq/asim_freq.sum()

### Calculate calibration constants
ao_summary_gq = base_share.rename({'share': 'census'}, axis=1)
ao_summary_gq['model'] = asim_share['weight']
ao_summary_gq['ratio'] = ao_summary_gq['model']/ao_summary_gq['census']

ao_summary_gq.to_csv('C:/abm_runs/rohans/calibration/auto_ownership/ao_summary_only_gq.csv')
ao_summary_gq

,census,model,ratio
auto_ownership,,,
0,0.054026,0.81154,15.021291
1,0.295868,0.18846,0.636972
2,0.398939,NaN,NaN
3,0.160256,NaN,NaN
4,0.090911,NaN,NaN


### Create or Read Crosswalk b/w MAZs and Tracts

In [8]:
if create_xwalk==1:
    ### Create a MAZ centroid geodataframe
    # maz_gdf['centroid'] = maz_gdf.geometry.centroid
    maz_gdf['centroid'] = maz_gdf.geometry.representative_point()
    maz_centroid_gdf = maz_gdf[['MGRA', 'centroid']].set_geometry('centroid').rename_geometry('geometry')

    ### Join MAZs to Tracts
    tract_maz_gdf = gpd.sjoin(tract_gdf[['NAME', 'GEOID', 'geometry']], maz_centroid_gdf, how='inner')
    tract_maz_gdf.rename({'NAME': 'Tract'}, axis=1, inplace=True)

    ### Crosswalk
    maz_xwalk_df = tract_maz_gdf[['MGRA', 'Tract', 'GEOID']]
    maz_xwalk_df.sort_values('MGRA', inplace=True)
    maz_xwalk_df['MGRA'] = maz_xwalk_df['MGRA'].astype(np.int64)
    maz_xwalk_df['Tract'] = maz_xwalk_df['Tract'].astype(float)
    maz_xwalk_df['GEOID'] = maz_xwalk_df['GEOID'].astype(np.int64)

    if len(tract_maz_gdf)==len(maz_gdf):
        maz_xwalk_df.to_csv(output_loc+'/'+xwalk_file_name, index=False)
    else:
        print('ERROR: number of MAZs in crosswalk differ from MAZs shape file')
else:
    maz_xwalk_df = pd.read_csv(xwalk_loc)

In [9]:
### Print xwalk to have a look
print('Xwalk Shape:', maz_xwalk_df.shape)
maz_xwalk_df.head(10)

Xwalk Shape: (24321, 3)


,MGRA,Tract,GEOID
0,1,27.05,6073002705
1,2,56.01,6073005601
2,3,154.07,6073015407
3,4,174.07,6073017407
4,5,174.07,6073017407
5,6,174.07,6073017407
6,7,74.02,6073007402
7,8,70.02,6073007002
8,9,133.21,6073013321
9,10,170.64,6073017064


In [10]:
### Count number of MAZs in each Tract
tract_maz_count_df = pd.pivot_table(maz_xwalk_df, 
                                    index=['Tract', 'GEOID'], 
                                    values='MGRA', 
                                    aggfunc='count').rename({'MGRA': 'MAZ_Count'}, axis=1)

tract_maz_count_df.sort_values('MAZ_Count', ascending=False)

,,MAZ_Count
Tract,GEOID,
209.03,6073020903,329
100.15,6073010015,258
209.04,6073020904,250
210.02,6073021002,232
171.06,6073017106,181
...,...,...
133.24,6073013324,4
91.09,6073009109,4
55.00,6073005500,4


In [11]:
hh_df

,household_id,home_zone_id,income,hhsize,HHT,auto_ownership,num_workers,bldgsz,sample_rate,income_in_thousands,income_segment,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_6_to_12,num_children_5_to_15,num_children_16_to_17,num_gradeschool,num_highschool,num_college_age,num_young_adults,num_predrive_child,num_nonworker_adults,num_university_students,num_fullTime_workers,num_partTime_workers,num_retired_adults,num_highschool_graduates,num_children_6_to_15,num_young_retirees,num_old_retirees,non_family,family,home_is_urban,home_is_rural,num_hh_in_zone,av_ownership,workplace_location_accessibility,shopping_accessibility,othdiscr_accessibility,weight
0,3,57,539038,1,4,1,0,3,0.235,539.038,4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,True,False,False,False,5,False,11.055949,12.082747,16.745117,4.255319
1,4,57,46849,8,1,1,3,7,0.235,46.849,2,5,3,3,5,1,3,4,0,4,1,0,2,4,1,0,1,1,0,3,4,0,0,False,True,False,False,5,False,10.744677,12.106373,16.210839,4.255319
2,8,57,53850,1,4,1,0,4,0.235,53.850,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,True,False,False,False,5,False,10.744677,12.106373,16.210839,4.255319
3,9,57,253095,2,1,2,1,6,0.235,253.095,4,1,2,2,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2,0,0,0,False,True,False,False,5,False,10.525013,11.448890,16.967464,4.255319
4,12,57,37156,1,4,1,0,1,0.235,37.156,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,True,False,False,False,5,False,10.744677,12.106373,16.210839,4.255319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272578,1160441,10222,278512,7,1,4,1,2,0.235,278.512,4,6,5,5,2,0,1,2,0,2,1,0,0,2,1,0,2,0,2,5,2,3,0,False,True,False,False,44,False,10.319807,14.322718,15.820274,4.255319
272579,1160442,10222,278512,7,1,4,1,2,0.235,278.512,4,6,5,5,2,0,1,2,0,2,1,0,0,2,1,0,2,0,2,5,2,3,0,False,True,False,False,44,False,10.319807,14.322718,15.820274,4.255319
272580,1160450,10222,39849,1,6,1,0,9,0.235,39.849,2,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,True,False,False,False,44,False,10.787433,14.347280,15.971924,4.255319
272581,1160458,10222,212169,3,1,3,3,2,0.235,212.169,4,0,3,3,0,0,0,0,0,0,0,1,0,0,0,0,3,0,0,3,0,0,0,False,True,False,False,44,False,10.319807,14.322718,15.820274,4.255319


In [12]:
### Merge Tract and GEOID to houeholds
hh_df = pd.merge(hh_df, maz_xwalk_df, how='left', left_on='home_zone_id', right_on='MGRA')

In [13]:
### Count households in each tract
tract_hh_count_df = pd.pivot_table(hh_df, index=['Tract', 'GEOID'], values='weight', aggfunc=np.sum).round()

### Calculate households in each tract by auto ownership
model_tract_ao_df = pd.pivot_table(hh_df, index=['Tract', 'GEOID'], columns=['auto_ownership'], values=['weight'], aggfunc=np.sum, fill_value=0).round()
model_tract_ao_df.columns = model_tract_ao_df.columns.get_level_values(1)
model_tract_ao_df.columns.names = [None]
model_tract_ao_df['total_model_hhs'] = tract_hh_count_df['weight']

### Add MAZ count of each tract
model_tract_ao_df['maz_count'] = tract_maz_count_df['MAZ_Count']

### Calculate share of AO in each tract
model_tract_ao_df[['model_auto_0_share', 'model_auto_1_share', 'model_auto_2_share', 'model_auto_3_share', 'model_auto_4_share']] = round(model_tract_ao_df[[0, 1, 2, 3, 4]].div(model_tract_ao_df['total_model_hhs'], axis=0)*100, 2)

### Column renaming and index resetting
model_tract_ao_df.rename({0: 'model_auto_0_hhs', 1: 'model_auto_1_hhs', 2: 'model_auto_2_hhs', 3: 'model_auto_3_hhs', 4: 'model_auto_4_hhs'}, axis=1, inplace=True)
model_tract_ao_df.reset_index(inplace=True)
model_tract_ao_df

,Tract,GEOID,model_auto_0_hhs,model_auto_1_hhs,model_auto_2_hhs,model_auto_3_hhs,model_auto_4_hhs,total_model_hhs,maz_count,model_auto_0_share,model_auto_1_share,model_auto_2_share,model_auto_3_share,model_auto_4_share
0,1.00,6073000100,17.0,404.0,515.0,149.0,72.0,1157.0,52,1.47,34.92,44.51,12.88,6.22
1,2.01,6073000201,111.0,566.0,396.0,51.0,38.0,1162.0,41,9.55,48.71,34.08,4.39,3.27
2,2.02,6073000202,166.0,1243.0,689.0,145.0,60.0,2302.0,57,7.21,54.00,29.93,6.30,2.61
3,3.01,6073000301,187.0,749.0,357.0,55.0,9.0,1357.0,19,13.78,55.20,26.31,4.05,0.66
4,3.02,6073000302,191.0,932.0,506.0,102.0,13.0,1745.0,26,10.95,53.41,29.00,5.85,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,218.00,6073021800,9.0,294.0,383.0,132.0,60.0,877.0,37,1.03,33.52,43.67,15.05,6.84
725,219.00,6073021900,64.0,264.0,328.0,115.0,34.0,804.0,125,7.96,32.84,40.80,14.30,4.23
726,220.00,6073022000,123.0,472.0,566.0,230.0,106.0,1498.0,18,8.21,31.51,37.78,15.35,7.08
727,221.01,6073022101,21.0,217.0,315.0,149.0,68.0,770.0,67,2.73,28.18,40.91,19.35,8.83


In [14]:
### Calculate Census Auto Ownership Share
acs_df[['Tract','County', 'State']] = acs_df['County'].str.split(', ', expand=True)
acs_df['Tract'] = acs_df['Tract'].str.split(' ', expand=True)[2].astype(float)
acs_df[['acs_auto_0_share', 'acs_auto_1_share', 'acs_auto_2_share', 'acs_auto_3_share', 'acs_auto_4_share']] = round(acs_df[['veh_0', 'veh_1', 'vhe_2', 'veh_3', 'veh_4']].div(acs_df['Total'], axis=0)*100, 2)
acs_df.loc[acs_df['Total']==0, ['acs_auto_0_share', 'acs_auto_1_share', 'acs_auto_2_share', 'acs_auto_3_share', 'acs_auto_4_share']] = 0

### Column renaming and index resetting
acs_df.rename({'Total': 'total_acs_hhs', 'veh_0': 'acs_auto_0_hhs', 'veh_1': 'acs_auto_1_hhs', 'veh_2': 'acs_auto_2_hhs', 'veh_3': 'acs_auto_3_hhs', 'veh_4': 'acs_auto_4_hhs'}, axis=1, inplace=True)
acs_df

,County,total_acs_hhs,acs_auto_0_hhs,acs_auto_1_hhs,vhe_2,acs_auto_3_hhs,acs_auto_4_hhs,Tract,State,acs_auto_0_share,acs_auto_1_share,acs_auto_2_share,acs_auto_3_share,acs_auto_4_share
0,San Diego County,1150,24,271,590,179,86,1.00,California,2.09,23.57,51.30,15.57,7.48
1,San Diego County,2343,59,1348,807,118,11,10.00,California,2.52,57.53,34.44,5.04,0.47
2,San Diego County,1121,70,187,472,197,195,100.01,California,6.24,16.68,42.11,17.57,17.40
3,San Diego County,1586,22,180,578,447,359,100.03,California,1.39,11.35,36.44,28.18,22.64
4,San Diego County,1105,28,146,323,226,382,100.04,California,2.53,13.21,29.23,20.45,34.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,San Diego County,1827,132,392,800,282,221,98.04,California,7.22,21.46,43.79,15.44,12.10
733,San Diego County,1842,61,683,640,277,181,98.05,California,3.31,37.08,34.74,15.04,9.83
734,San Diego County,4,0,4,0,0,0,99.01,California,0.00,100.00,0.00,0.00,0.00
735,San Diego County,0,0,0,0,0,0,99.02,California,0.00,0.00,0.00,0.00,0.00


In [15]:
### Merge ACS and Model data
tract_ao_df = pd.merge(model_tract_ao_df[['Tract', 'GEOID', 'total_model_hhs', 'maz_count', 'model_auto_0_hhs', 'model_auto_0_share']], 
                       acs_df[['Tract', 'total_acs_hhs', 'acs_auto_0_hhs', 'acs_auto_0_share']], 
                       how='left', 
                       left_on='Tract', 
                       right_on='Tract')
tract_ao_df['auto_0_count_diff'] = tract_ao_df['model_auto_0_hhs'] - tract_ao_df['acs_auto_0_hhs']
tract_ao_df['auto_0_count_diff(%)'] = round(tract_ao_df['auto_0_count_diff']/tract_ao_df['acs_auto_0_hhs']*100, 1)
tract_ao_df['auto_0_count_diff(%)'].fillna(0, inplace=True)
tract_ao_df['auto_0_share_diff'] = tract_ao_df['model_auto_0_share'] - tract_ao_df['acs_auto_0_share']
tract_ao_df

,Tract,GEOID,total_model_hhs,maz_count,model_auto_0_hhs,model_auto_0_share,total_acs_hhs,acs_auto_0_hhs,acs_auto_0_share,auto_0_count_diff,auto_0_count_diff(%),auto_0_share_diff
0,1.00,6073000100,1157.0,52,17.0,1.47,1150,24,2.09,-7.0,-29.2,-0.62
1,2.01,6073000201,1162.0,41,111.0,9.55,1187,120,10.11,-9.0,-7.5,-0.56
2,2.02,6073000202,2302.0,57,166.0,7.21,2273,132,5.81,34.0,25.8,1.40
3,3.01,6073000301,1357.0,19,187.0,13.78,1339,130,9.71,57.0,43.8,4.07
4,3.02,6073000302,1745.0,26,191.0,10.95,1625,41,2.52,150.0,365.9,8.43
...,...,...,...,...,...,...,...,...,...,...,...,...
724,218.00,6073021800,877.0,37,9.0,1.03,800,10,1.25,-1.0,-10.0,-0.22
725,219.00,6073021900,804.0,125,64.0,7.96,699,76,10.87,-12.0,-15.8,-2.91
726,220.00,6073022000,1498.0,18,123.0,8.21,1601,261,16.30,-138.0,-52.9,-8.09
727,221.01,6073022101,770.0,67,21.0,2.73,889,0,0.00,21.0,inf,2.73


In [16]:
### Save the tract level summary to csv
save_df = tract_ao_df[['Tract', 'GEOID', 'maz_count', 
                       'total_model_hhs', 'model_auto_0_hhs', 'model_auto_0_share', 
                       'total_acs_hhs', 'acs_auto_0_hhs', 'acs_auto_0_share', 
                       'auto_0_count_diff', 'auto_0_count_diff(%)', 'auto_0_share_diff']]
save_df.to_csv(output_loc+'/sandag_tract_ao_table (without gq).csv', index=False)

In [17]:
### Add geometry and convert dataframe to geodataframe
tract_gdf['GEOID'] = tract_gdf['GEOID'].astype(np.int64)
tract_ao_gdf = pd.merge(tract_ao_df, tract_gdf[['GEOID', 'geometry']], on='GEOID')
tract_ao_gdf = gpd.GeoDataFrame(tract_ao_gdf)
tract_ao_gdf

,Tract,GEOID,total_model_hhs,maz_count,model_auto_0_hhs,model_auto_0_share,total_acs_hhs,acs_auto_0_hhs,acs_auto_0_share,auto_0_count_diff,auto_0_count_diff(%),auto_0_share_diff,geometry
0,1.00,6073000100,1157.0,52,17.0,1.47,1150,24,2.09,-7.0,-29.2,-0.62,"POLYGON ((-117.19490 32.75278, -117.19471 32.7..."
1,2.01,6073000201,1162.0,41,111.0,9.55,1187,120,10.11,-9.0,-7.5,-0.56,"POLYGON ((-117.17887 32.75765, -117.17797 32.7..."
2,2.02,6073000202,2302.0,57,166.0,7.21,2273,132,5.81,34.0,25.8,1.40,"POLYGON ((-117.18404 32.74571, -117.18383 32.7..."
3,3.01,6073000301,1357.0,19,187.0,13.78,1339,130,9.71,57.0,43.8,4.07,"POLYGON ((-117.16864 32.74897, -117.16840 32.7..."
4,3.02,6073000302,1745.0,26,191.0,10.95,1625,41,2.52,150.0,365.9,8.43,"POLYGON ((-117.16400 32.74091, -117.16400 32.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,218.00,6073021800,877.0,37,9.0,1.03,800,10,1.25,-1.0,-10.0,-0.22,"POLYGON ((-117.19351 32.68665, -117.19306 32.6..."
725,219.00,6073021900,804.0,125,64.0,7.96,699,76,10.87,-12.0,-15.8,-2.91,"POLYGON ((-117.13746 32.67843, -117.13542 32.6..."
726,220.00,6073022000,1498.0,18,123.0,8.21,1601,261,16.30,-138.0,-52.9,-8.09,"POLYGON ((-117.09190 32.68388, -117.09169 32.6..."
727,221.01,6073022101,770.0,67,21.0,2.73,889,0,0.00,21.0,inf,2.73,"POLYGON ((-117.33371 33.14433, -117.33365 33.1..."


In [18]:
tract_ao_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [19]:
### Sample custom colormap
custom_colorscale = folium.LinearColormap(['blue', 'white', 'red'], vmin=-20, vmax=20)
custom_colorscale

In [20]:
### Function to create colormap
def generate_choropleth_map(
        gdf, # gpd.GeoDataFrame that you want to display
        output_map_dir, # directory to save the HTML file
        filename, # name of the HTML file
        colorscale_caption, # caption to be displayed below colorscale
        colorscale_list=['green', 'yellow', 'red'], # list of colors to use in colorscale in sequence,
        fill_column='TAZ_Mean', # column or values used to create the colormap
        tooltip_list=['TAZ', 'TAZ_Min', 'TAZ_Max', 'TAZ_Mean', 'Global_Min', 'Global_Max', 'Global_Mean'], # statistics to display in tooltip
        step_colormap=False, # option to change to a step colormap instead of the default linear colormap
        num_color_steps=7, # preferred number of steps in the colormap
        cap_upper_limit=None, # upper limit when capping the fill_column
        cap_lower_limit=None, # lower limit when capping the fill_column
        coordinates=[33.043, -116.756], # initial co-ordinates of map, SANDAG
        tile_layer='stamentoner', # base folium layer
        tile_name='stamentoner', # title for base layer
        fillOpacity=0.75, # opacity of the choropleth layer over base map
        save_map=True): # option to save map as an HTML
    
    ### Create a base Folium map for region specified in coordinates
    map = folium.Map(location=coordinates, zoom_start=10, tiles=None, overlay=False) 

    ### Adding tiles as a separate layer allows for more control and more customization
    folium.TileLayer(tile_layer, name=tile_name, control=False).add_to(map)

    ### Cap lower and upper limits of the fill_column
    gdf[fill_column + '_capped'] = gdf[fill_column].clip(lower=cap_lower_limit, upper=cap_upper_limit)

    ### Create a custom color scale
    if cap_lower_limit is not None:
        scale_min = cap_lower_limit
    else:
        scale_min = np.floor(gdf[fill_column + '_capped'].min())
    
    if cap_upper_limit is not None:
        scale_max = cap_upper_limit
    else:
        scale_max = np.ceil(gdf[fill_column + '_capped'].max())

    colormap = folium.LinearColormap(
        colorscale_list, 
        vmin=scale_min, 
        vmax=scale_max,
        caption=colorscale_caption
        )
    
    ### Change linear colormap to step colormap
    if step_colormap:
        colormap = colormap.to_step(
            n=num_color_steps,
            data=np.percentile(gdf[fill_column + '_capped'], np.arange(0, 100+1, (100)/num_color_steps)),
            method='quantiles',
            round_method='int'
        )

    ### Add the colormap as a legend
    colormap.add_to(map)

    ### Creating a function that would call on these values
    style_function = lambda x: {"weight": 0, 
                                'color': 'black',
                                'fillColor': colormap(x['properties'][fill_column + '_capped']), 
                                'fillOpacity': fillOpacity}
    highlight_function = lambda x: {'weight': 3,
                                    'fillColor': colormap(x['properties'][fill_column + '_capped'])}

    ### Add the colormap
    folium.features.GeoJson(
        data=gdf,
        style_function=style_function,
        tooltip=folium.features.GeoJsonTooltip(tooltip_list),
        highlight_function=highlight_function
    ).add_to(map) 

    ### Export map as HTML
    if save_map:
        map.save(output_map_dir + "/" + filename + ".html")

    ### Return the map so other changes can be made later as needed
    return map

In [21]:
_ = generate_choropleth_map(gdf=tract_ao_gdf.rename({'model_auto_0_share': 'model_auto_0_share(%)', 'acs_auto_0_share': 'acs_auto_0_share(%)'}, axis=1), 
                            output_map_dir=output_loc, 
                            filename='sandag_tract_ao_choropleth_map-count (without gq)', 
                            colorscale_caption='Percent Difference of Zero Auto Household Count', 
                            colorscale_list=['blue', 'white', 'red'],
                            fill_column='auto_0_count_diff(%)', 
                            cap_lower_limit=-100,
                            cap_upper_limit=100,
                            tooltip_list=['Tract', 'GEOID', 'maz_count', 'total_model_hhs', 'total_acs_hhs', 'model_auto_0_hhs', 'acs_auto_0_hhs', 'auto_0_count_diff(%)', 'model_auto_0_share(%)', 'acs_auto_0_share(%)', 'auto_0_share_diff'])

In [22]:
_ = generate_choropleth_map(gdf=tract_ao_gdf.rename({'model_auto_0_share': 'model_auto_0_share(%)', 'acs_auto_0_share': 'acs_auto_0_share(%)'}, axis=1), 
                            output_map_dir=output_loc, 
                            filename='sandag_tract_ao_choropleth_map-share (without gq)', 
                            colorscale_caption='Difference of Zero Auto Share Percent', 
                            colorscale_list=['blue', 'white', 'red'],
                            fill_column='auto_0_share_diff', 
                            cap_lower_limit=-10,
                            cap_upper_limit=10,
                            tooltip_list=['Tract', 'GEOID', 'maz_count', 'total_model_hhs', 'total_acs_hhs', 'model_auto_0_hhs', 'acs_auto_0_hhs', 'auto_0_count_diff(%)', 'model_auto_0_share(%)', 'acs_auto_0_share(%)', 'auto_0_share_diff'])

In [23]:
end_time = datetime.datetime.now()
print("Start Time:", start_time)
print("End Time:", end_time)
print_runtime(start_time, end_time)

Start Time: 2023-07-17 00:12:28.878099
End Time: 2023-07-17 00:12:51.161385
Run Time: 0.0 hrs 0.0 mins 22 sec
